TODO: write heading

In [ ]:
#TODO: add imports

In [ ]:
#TODO: extract features from images
# using scikit-image instead of cv2

In [ ]:
#TODO: retrieve training and test data 
# (shoudln't be that different from the original one)

Section 1: randomly picked N elements approach

(we decide to label 10% of the unlabled images we have. we pick randomly which ones to label)

In [ ]:
#TODO: show off how poorly perform a randomly choosen training 
# like the old one but with less labeled samples

Section 2: active learning approach with N labeled elements

(we decide to label 10% of the unlabled images we have. we pick which ones to label using active learning)

In [ ]:
#TODO: repeat the same stuff above BUT with an active learning 
# loop to show off much better performances

Section 3: original model (traditional but with all samples)

(we decide to label all the unlabled images we have)

In [ ]:
#TODO: almost a copy of original steps 
# to show that the active learning approach achieves similar 
# results but with less data

Section 4: fancy graphs to prove our thesis